In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.python.ops.gen_batch_ops import Batch

2024-10-16 15:06:53.976588: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 15:06:53.997244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 15:06:53.997263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 15:06:53.997278: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 15:06:54.001235: I tensorflow/core/platform/cpu_feature_g

In [2]:
X_train = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_X_train.csv')
X_test = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_X_test.csv')
X_val = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_X_val.csv')
y_train = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_y_train.csv')
y_test = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_y_test.csv')
y_val = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/5_latent_y_val.csv')

X_train.drop('Unnamed: 0', axis = 1, inplace = True)
X_test.drop('Unnamed: 0', axis = 1, inplace = True)
X_val.drop('Unnamed: 0', axis = 1, inplace = True)
y_train.drop('Unnamed: 0', axis = 1, inplace = True)
y_test.drop('Unnamed: 0', axis = 1, inplace = True)
y_val.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of X_val: {X_val.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')
print(f'Shape of y_val: {y_val.shape}')

Shape of X_train: (203, 5)
Shape of X_test: (60, 5)
Shape of X_val: (27, 5)
Shape of y_train: (203, 1)
Shape of y_test: (60, 1)
Shape of y_val: (27, 1)


In [4]:
total_samples = X_train.shape[0] + X_test.shape[0] + X_val.shape[0]
print(f'Train split: {np.round(((X_train.shape[0] / total_samples) * 100), 2)}%')
print(f'Validation split: {np.round(((X_val.shape[0] / total_samples) * 100), 2)}%')
print(f'Test split: {np.round(((X_test.shape[0] / total_samples) * 100), 2)}%')

Train split: 70.0%
Validation split: 9.31%
Test split: 20.69%


In [5]:
# ANN binary classifier
from tensorflow.keras.regularizers import l1_l2
input_layer_1 = Input(shape = (X_train.shape[1])) # 2
x = Dense(8, activation = 'relu')(input_layer_1)
x = Dropout(0.3)(x)
x = Dense(4, activation = 'relu')(x)
x = Dropout(0.3)(x)
classifier_layer_1 = Dense(1, activation = 'sigmoid')(x)
ann_classifier_model_1 = Model(input_layer_1, classifier_layer_1)

'''input_layer = Input(shape = (X_train.shape[1])) # 3
x = Dense(256, activation = 'relu')(input_layer)
x = BatchNormalization(momentum = 0.8)(x)
x = Dropout(0.2)(x)
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)
classifier_layer = Dense(1, activation = 'sigmoid')(x)
ann_classifier_model = Model(input_layer, classifier_layer)'''

input_layer_2 = Input(shape = (X_train.shape[1]))
y = Dense(32, activation = 'relu')(input_layer_2)
y = Dropout(0.1)(y)
y = Dense(16, activation = 'relu')(y)
y = Dropout(0.25)(y)
y = Dense(8, activation = 'relu')(y)
y = Dropout(0.5)(y)
y = Dense(4, activation = 'relu')(y)
classifier_layer_2 = Dense(1, activation = 'sigmoid')(y)
ann_classifier_model_2 = Model(input_layer_2, classifier_layer_2)
'''input_layer = Input(shape = (X_train.shape[1]))
x = Dense(256, activation = 'relu')(input_layer)
x = BatchNormalization(momentum = 0.8)(x)
x = Dropout(0.1)(x)
x = Dense(128, activation = 'relu')(x)
classifier_layer = Dense(1, activation = 'sigmoid')(x)
ann_classifier_model = Model(input_layer, classifier_layer)'''

2024-10-16 15:06:55.088176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 15:06:55.125393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 15:06:55.125520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

"input_layer = Input(shape = (X_train.shape[1]))\nx = Dense(256, activation = 'relu')(input_layer)\nx = BatchNormalization(momentum = 0.8)(x)\nx = Dropout(0.1)(x)\nx = Dense(128, activation = 'relu')(x)\nclassifier_layer = Dense(1, activation = 'sigmoid')(x)\nann_classifier_model = Model(input_layer, classifier_layer)"

**Experimentations Done:**

1. ANN with I/P layer, Dense_layer(128, relu) and classifier layer val_acc = 69%, train_acc = 70%
2. Dense_layer(256, relu) + 1 val_acc = 70.37%, train_acc = 67.49%.
3. 

In [6]:
from tensorflow.keras.losses import BinaryCrossentropy
ann_classifier_model_1.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
ann_classifier_model_2.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
lrs = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.05, patience = 10, min_lr = 1e-4)
es = EarlyStopping(monitor = 'val_loss', patience = 10)

In [8]:
tf.random.set_seed(42)
history_ann_1 = ann_classifier_model_1.fit(X_train, y_train, validation_data = (X_val, y_val), callbacks = [lrs, es], epochs = 200)


Epoch 1/200


2024-10-16 15:06:55.746899: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-16 15:06:56.416075: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x71164435b0b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-16 15:06:56.416094: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-10-16 15:06:56.418633: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-16 15:06:56.426251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-10-16 15:06:56.475216: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 [==============================] - 1s 21ms/step - loss: 0.7849 - accuracy: 0.5025 - val_loss: 0.7704 - val_accuracy: 0.5185 - lr: 0.0010
Epoch 2/200
7/7 [==============================] - 0s 7ms/step - loss: 0.7650 - accuracy: 0.4926 - val_loss: 0.7598 - val_accuracy: 0.5185 - lr: 0.0010
Epoch 3/200
7/7 [==============================] - 0s 7ms/step - loss: 0.7604 - accuracy: 0.4532 - val_loss: 0.7527 - val_accuracy: 0.5185 - lr: 0.0010
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 0.7871 - accuracy: 0.4680 - val_loss: 0.7464 - val_accuracy: 0.5185 - lr: 0.0010
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 0.7470 - accuracy: 0.5025 - val_loss: 0.7408 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 6/200
7/7 [==============================] - 0s 5ms/step - loss: 0.7493 - accuracy: 0.4532 - val_loss: 0.7365 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 0.7289 - accuracy: 0.5025 - 

In [9]:
history_ann_2 = ann_classifier_model_2.fit(X_train, y_train, validation_data = (X_val, y_val), callbacks = [lrs, es], epochs = 200)

Epoch 1/200
7/7 [==============================] - 1s 16ms/step - loss: 0.7032 - accuracy: 0.4778 - val_loss: 0.6947 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 2/200
7/7 [==============================] - 0s 5ms/step - loss: 0.6942 - accuracy: 0.5074 - val_loss: 0.6944 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 3/200
7/7 [==============================] - 0s 6ms/step - loss: 0.6960 - accuracy: 0.5172 - val_loss: 0.6941 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 4/200
7/7 [==============================] - 0s 5ms/step - loss: 0.6948 - accuracy: 0.5320 - val_loss: 0.6937 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6886 - accuracy: 0.5123 - val_loss: 0.6934 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 6/200
7/7 [==============================] - 0s 6ms/step - loss: 0.6903 - accuracy: 0.5025 - val_loss: 0.6934 - val_accuracy: 0.4815 - lr: 0.0010
Epoch 7/200
7/7 [==============================] - 0s 9ms/step - loss: 0.6913 - accurac

In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_test, tf.round(ann_classifier_model_1.predict(X_test))))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.53      0.27      0.36        33
           1       0.44      0.70      0.54        27

    accuracy                           0.47        60
   macro avg       0.49      0.49      0.45        60
weighted avg       0.49      0.47      0.44        60



In [11]:
print(classification_report(y_test, tf.round(ann_classifier_model_2.predict(X_test))))

2/2 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        33
           1       0.45      1.00      0.62        27

    accuracy                           0.45        60
   macro avg       0.23      0.50      0.31        60
weighted avg       0.20      0.45      0.28        60



/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

In [12]:
#Using Logistic regression as meta learner
y_pred_1 = ann_classifier_model_1.predict(X_val)
y_pred_2 = ann_classifier_model_2.predict(X_val)
combined_preds = np.column_stack((y_pred_1, y_pred_2))

1/1 [==============================] - 0s 12ms/step


In [13]:
#Maxvoting
combined_preds

array([[0.514397  , 0.5021219 ],
       [0.59285146, 0.5141195 ],
       [0.57653975, 0.51457626],
       [0.49483097, 0.5003954 ],
       [0.53136015, 0.50717586],
       [0.530031  , 0.50537395],
       [0.54245484, 0.5086736 ],
       [0.43934622, 0.5003954 ],
       [0.4990827 , 0.5003954 ],
       [0.591113  , 0.51645637],
       [0.4771995 , 0.502283  ],
       [0.4710511 , 0.5003954 ],
       [0.529516  , 0.503851  ],
       [0.5682143 , 0.51584315],
       [0.6176096 , 0.52148163],
       [0.5248875 , 0.5117376 ],
       [0.5232454 , 0.5039897 ],
       [0.5483189 , 0.50447077],
       [0.5181044 , 0.50341463],
       [0.5868371 , 0.52189916],
       [0.59015226, 0.5177743 ],
       [0.5643534 , 0.5260496 ],
       [0.5558926 , 0.515542  ],
       [0.5048943 , 0.5024469 ],
       [0.4627933 , 0.5007988 ],
       [0.59543645, 0.5257413 ],
       [0.5281956 , 0.5077129 ]], dtype=float32)

In [14]:
y_pred_1.shape

(27, 1)

In [15]:
col_dict = {'model_1': tf.squeeze(y_pred_1), 'model_2': tf.squeeze(y_pred_2)}
preds_df = pd.DataFrame(col_dict)

In [16]:
preds_df

,model_1,model_2
0,0.514397,0.502122
1,0.592851,0.514120
2,0.576540,0.514576
3,0.494831,0.500395
4,0.531360,0.507176
5,0.530031,0.505374
6,0.542455,0.508674
7,0.439346,0.500395
8,0.499083,0.500395
9,0.591113,0.516456


In [17]:
preds_df['final_preds'] = (preds_df['model_1'] + preds_df['model_2'])/2

In [18]:
preds_df

,model_1,model_2,final_preds
0,0.514397,0.502122,0.508259
1,0.592851,0.514120,0.553486
2,0.576540,0.514576,0.545558
3,0.494831,0.500395,0.497613
4,0.531360,0.507176,0.519268
5,0.530031,0.505374,0.517702
6,0.542455,0.508674,0.525564
7,0.439346,0.500395,0.469871
8,0.499083,0.500395,0.499739
9,0.591113,0.516456,0.553785


In [19]:
preds_df['labels'] = preds_df['final_preds'].apply(lambda x: 0 if x <= 0.5 else 1)

In [20]:
preds_df

,model_1,model_2,final_preds,labels
0,0.514397,0.502122,0.508259,1
1,0.592851,0.514120,0.553486,1
2,0.576540,0.514576,0.545558,1
3,0.494831,0.500395,0.497613,0
4,0.531360,0.507176,0.519268,1
5,0.530031,0.505374,0.517702,1
6,0.542455,0.508674,0.525564,1
7,0.439346,0.500395,0.469871,0
8,0.499083,0.500395,0.499739,0
9,0.591113,0.516456,0.553785,1


In [21]:
print(classification_report(y_val, preds_df['labels']))

              precision    recall  f1-score   support

           0       0.83      0.36      0.50        14
           1       0.57      0.92      0.71        13

    accuracy                           0.63        27
   macro avg       0.70      0.64      0.60        27
weighted avg       0.71      0.63      0.60        27



**Running tests on the 13 latent features extracted without applying PCA**

In [22]:
X_13 = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/13_latent_X.csv')
y_13 = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/Gleason_Scores.csv')